In [ ]:
!pip install -q transformers datasets peft accelerate bitsandbytes sentencepiece


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving combined_train.json to combined_train.json


In [ ]:
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    BitsAndBytesConfig,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

MODEL_NAME = "google/flan-t5-base"
OUTPUT_DIR = "/content/flan_lora_adapter"

# 4-bit configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

model = AutoModelForSeq2SeqLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto"
)

model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)

model = get_peft_model(model, lora_config)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

dataset = load_dataset("json", data_files="combined_train.json")["train"]
dataset = dataset.train_test_split(test_size=0.1)

def preprocess(batch):
    inputs = [
        inst + "\n\n" + inp
        for inst, inp in zip(batch["instruction"], batch["input"])
    ]
    model_inputs = tokenizer(inputs, truncation=True, max_length=512)
    labels = tokenizer(batch["output"], truncation=True, max_length=512)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

dataset = dataset.map(
    preprocess,
    batched=True,
    remove_columns=dataset["train"].column_names
)

training_args = Seq2SeqTrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=2,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    learning_rate=2e-4,
    logging_steps=20,
    save_strategy="epoch",
    fp16=True,
    report_to="none"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=DataCollatorForSeq2Seq(tokenizer, model=model),
)

trainer.train()

trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print("LoRA adapter saved.")


Loading weights:   0%|          | 0/282 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


LoRA adapter saved.


In [ ]:
from peft import PeftModel

BASE_MODEL = "google/flan-t5-base"
MERGED_MODEL_DIR = "/content/Interview_Model_Final"

# Load base model in full precision
base_model = AutoModelForSeq2SeqLM.from_pretrained(BASE_MODEL)

# Load trained adapter
model = PeftModel.from_pretrained(base_model, OUTPUT_DIR)

# Merge LoRA weights into base model
model = model.merge_and_unload()

# Save merged standalone model
model.save_pretrained(MERGED_MODEL_DIR)
tokenizer.save_pretrained(MERGED_MODEL_DIR)

print("Standalone merged model saved.")


Loading weights:   0%|          | 0/282 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Standalone merged model saved.


In [ ]:
import shutil
import os

FINAL_DRIVE_PATH = "/content/drive/MyDrive/Interview_Model_Final"

if os.path.exists(FINAL_DRIVE_PATH):
    shutil.rmtree(FINAL_DRIVE_PATH)

shutil.copytree(MERGED_MODEL_DIR, FINAL_DRIVE_PATH)

print("Model copied to Google Drive.")


Model copied to Google Drive.


In [ ]:
zip_path = "/content/drive/MyDrive/Interview_Model_Final.zip"

shutil.make_archive(
    "/content/drive/MyDrive/Interview_Model_Final",
    'zip',
    FINAL_DRIVE_PATH
)

print("Zipped model saved at:", zip_path)


Zipped model saved at: /content/drive/MyDrive/Interview_Model_Final.zip
